##**Question 1: [IPO] Withdrawn IPOs by Company Type**

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list (stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

Steps:

    1. Use pandas.read_html() with the URL above to load the IPO withdrawal table into a DataFrame. It is a similar process to Code Snippet 1 discussed at the livestream. You should get 99 entries.

    2. Create a new column called Company Class, categorizing company names based on patterns like:

        ◦ “Acquisition Corp” or “Acquisition Corporation” → Acq.Corp
        ◦ “Inc” or “Incorporated” → Inc
        ◦ “Group” → Group
        ◦ “Ltd” or “Limited” → Limited
        ◦ “Holdings” → Holdings
        ◦ Others → Other

    • Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).
    • Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

    3. Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column). Examples:

        ◦ '$8.00-$10.00' → 9.0
        ◦ '$5.00' → 5.0
        ◦ '-' → None

    4. Convert Shares Offered to numeric, clean missing or invalid values.
    5. Create a new column:
       Withdrawn Value = Shares Offered * Avg Price (71 non-null values)
    6. Group by Company Class and calculate total withdrawn value.
    7. Answer: Which class had the highest total value of withdrawals?


In [ ]:
# 1. Use pandas.read_html() to load the IPO withdrawal table into a DataFrame
import pandas as pd
import requests
from io import StringIO

def get_withdrawn_ipos(url: str) -> pd.DataFrame:
    """
    Fetch IPO data from a given URL.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found at {url}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
    except ValueError as ve:
        print(f"Data error for {url}: {ve}")
    except Exception as ex:
        print(f"Unexpected error for {url}: {ex}")

    return pd.DataFrame()

In [ ]:
get_withdrawn_ipos = get_withdrawn_ipos("https://stockanalysis.com/ipos/withdrawn/")

In [ ]:
withdrawn_ipos = get_withdrawn_ipos.copy()
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [ ]:
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [ ]:
withdrawn_ipos.tail()

,Symbol,Company Name,Price Range,Shares Offered
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000


In [ ]:
print(withdrawn_ipos['Company Name'].iloc[0])
print(type(withdrawn_ipos['Company Name'].iloc[0]))

Odyssey Therapeutics, Inc.
<class 'str'>


In [ ]:
#2. Create a new column called Company Class, categorizing company names based on patterns
import string

def categorize_company(name: str) -> str: # Takes a company name as input
    """
    Categorizes company names based on specific patterns.
    Args:
        name: The company name string.
    Returns:
        The categorized company class.
    """
    if not isinstance(name, str):# Handle non-string input
        return 'Other'
    # Convert to lowercase, remove punctuation and split into a list of individual words
    to_low = name.lower()
    remove_punct = to_low.translate(str.maketrans('', '', string.punctuation))
    words = remove_punct.split()
    # Check for patterns in order and return the class
    if "acquisition" in words and ("corp." in words or "corporation" in words):
        return 'Acq.Corp'
    elif "inc" in words or "incorporated" in words:
        return 'Inc'
    elif "group" in words:
        return 'Group'
    elif "ltd" in words or "limited" in words:
        return 'Limited'
    elif "holdings" in words:
        return 'Holdings'
    else:
        return 'Other'

In [ ]:
withdrawn_ipos['Company Class'] = withdrawn_ipos['Company Name'].apply(categorize_company)
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other


In [ ]:
# 3. Define a new field Avg. price by parsing the Price Range field
import numpy as np

def avg_price_field(price_range: str) -> float or None:
    """
    Parses a price range string and calculates the average price.
    Args:
        price_range: The string from the 'Price Range' column.
    Returns:
        The average price as a float, or None if the input is '-' or invalid.
    """
    if not isinstance(price_range, str):
        return None  # Handle non-string inputs
    if price_range == '-':
        return None  # Handle the '-' case

    # Remove the dollar sign
    price_range_clean = price_range.replace('$', '')

    try:
        if '-' in price_range_clean:
            # Split the range and calculate the average
            low_price, high_price = price_range_clean.split('-')
            avg_price = (float(low_price) + float(high_price)) / 2
            return avg_price
        else:
            # Single price
            return float(price_range_clean)
    except ValueError:
        # Handle cases where parsing fails (e.g., unexpected format)
        return None

In [ ]:
withdrawn_ipos['Avg. price'] = withdrawn_ipos['Price Range'].apply(avg_price_field)

In [ ]:
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN


In [ ]:
# 4. Convert Shares Offered to numeric, clean missing or invalid values
def proc_shares_offered(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans and converts the 'Shares Offered' column to numeric.
    Removes commas and handles missing/invalid values by converting them to NaN.
    Args:
        df: The pandas DataFrame with a 'Shares Offered' column.
    Returns:
        The DataFrame with a new numeric column 'Shares Offered_numeric'.
    """
    # Clean the 'Shares Offered' column
    # Remove commas, strip whitespace, and replace empty strings with NaN
    df['Shares Offered'] = df['Shares Offered'] \
        .astype(str) \
        .str.replace(',', '', regex=False) \
        .str.strip() \
        .replace('', np.nan)

    # Convert to numeric, coercing errors to NaN
    df['Shares Offered'] = pd.to_numeric(
        df['Shares Offered'], errors='coerce'
    )

    return df

In [ ]:
preproc_shares_offered = proc_shares_offered(withdrawn_ipos)

In [ ]:
withdrawn_ipos = preproc_shares_offered.copy()

In [ ]:
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN
4,ONE,One Power Company,-,NaN,Other,NaN


In [ ]:
# 5. Create a new column: Withdrawn Value
# The result of multiplication with NaN's will also be a NaN
withdrawn_ipos['Withdrawn Value'] = withdrawn_ipos['Shares Offered'] * withdrawn_ipos['Avg. price']

In [ ]:
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN,NaN
4,ONE,One Power Company,-,NaN,Other,NaN,NaN


In [ ]:
# 6. Group by 'Company Class' and calculate the sum of 'Withdrawn Value'

# transform('sum') calculates the sum for each group and "broadcasts" the sum back to every row within a group
withdrawn_ipos['Total Withdrawn Value by Class'] = withdrawn_ipos.groupby('Company Class')['Withdrawn Value'].transform('sum')

In [ ]:
withdrawn_ipos.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value,Total Withdrawn Value by Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN,NaN,2.257164e+09
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0,2.257164e+09
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN,NaN,2.257164e+09
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN,NaN,2.257164e+09
4,ONE,One Power Company,-,NaN,Other,NaN,NaN,3.613920e+09


In [ ]:
withdrawn_ipos.tail()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value,Total Withdrawn Value by Class
95,FHP,"Freehold Properties, Inc.",-,NaN,Inc,NaN,NaN,2.257164e+09
96,CHO,Chobani Inc.,-,NaN,Inc,NaN,NaN,2.257164e+09
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231.0,Inc,19.5,600000004.5,2.257164e+09
98,GLGX,"Gerson Lehrman Group, Inc.",-,NaN,Inc,NaN,NaN,2.257164e+09
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000.0,Other,18.5,300070000.0,3.613920e+09


In [ ]:
# 7: Answer: Which class had the highest total value of withdrawals?

# Find the maximum value the index in 'Total Withdrawn Value by Class'
index_of_max_value = withdrawn_ipos['Total Withdrawn Value by Class'].idxmax()
print(index_of_max_value)

# Get the index from the 'Company Class' column in that same row
class_with_highest_total = withdrawn_ipos.loc[index_of_max_value, 'Company Class']
print(class_with_highest_total)

4
Other


##**Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)**

**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale yfinance OHLCV data download and perform basic financial calculations on IPO stocks.

Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:

  https://stockanalysis.com/ipos/2024/
  Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024).
  ➤ You should have 75 tickers.

2. Use Code Snippet 7 to download daily stock data for those tickers (via yfinance).
Make sure you understand how growth_1d ... growth_365d, and volatility columns are defined.
Define a new column growth_252d representing growth after 252 trading days (~1 year), in addition to any other growth periods you already track.

3. Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:

  `stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']`

  ⚠️ IMPORTANT Please use the original version of annualized volatility calculation (it was later corrected to another formula):

  `stocks_df['volatility'] = stocks_df['Close'].rolling(30).std() * np.sqrt(252)`

4. Filter the DataFrame to keep data only for the trading day:
‘2025-06-06’

  Compute descriptive statistics (e.g., .describe()) for these columns:

  * growth_252d
  * Sharpe

  You should observe:

  * growth_252d is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
  * Median growth_252d is approximately 0.75 (indicating a 25% decline), while mean is about 1.15, showing a bias towards high-growth companies pushing the average up.

5. Answer:

  * What is the median Sharpe ratio for these 71 stocks?
  * Note: Positive Sharpe means growth exceeding the risk-free rate of 4.5%.
  * [Additional] Do you observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe?

In [ ]:
# Using the same approach as in Question 1, download the IPOs in 2024 from (keep only those IPOs before 1 June 2024)
from datetime import date

def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:
# Download IPO data for 2024
ipos_2024 = get_ipos_by_year(2024)

# Convert 'IPO Date' to datetime objects
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], format='mixed')

# Filter to keep only IPOs before June 1, 2024
ipos_2024_filtered = ipos_2024[ipos_2024['IPO Date'] < '2024-06-01']

# Print the number of tickers after filtering
print(f"Number of tickers after filtering: {len(ipos_2024_filtered)}")

Number of tickers after filtering: 77


In [ ]:
ipos_2024_filtered

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.22,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.49,114.65%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$18.70,-4.40%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.60,6.00%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.26,-93.40%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.07,-82.17%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.50,-11.85%
222,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.07,-48.25%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$18.26,-13.05%


In [ ]:
ipos_2024_filtered[ipos_2024_filtered['IPO Price'].astype(str).str.contains('-')]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.22,-
174,2024-04-17,SUPX,SuperX AI Technology Limited,-,$9.79,-


In [ ]:
ipos_2024_filtered = ipos_2024_filtered[~ipos_2024_filtered['IPO Price'].astype(str).str.contains('-')]

In [1]:
# Print the number of tickers after filtering
print(len(ipos_2024_filtered))

75


In [ ]:
# 2. Use Code Snippet 7 to download daily stock data for those tickers (via yfinance)
import yfinance as yf
import time

ALL_TICKERS = ipos_2024_filtered['Symbol'].tolist()
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # Generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # Historical returns
  for i in [1,3,7,30,90,252,365]: # Added 252 for 252-day growth
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  # How much did the stock price change over the next 30 days relative to today's price?
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close'] # Shifts the values upwards by 30 rows in the future

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  # Multiply by the square root of the number of periods in a year to scale a measure of volatility from a shorter period (30 days) to an annual period.
  historyPrices['volatility'] = historyPrices['Close'].rolling(30).std() * np.sqrt(252) # Statistical convention on the assumption that price changes are independent and random.

  # Predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # Sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [ ]:
stocks_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_90d,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.092017,NaN,NaN,0,0.089496,NaN,1
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,0.998054,NaN,NaN,0,0.084436,NaN,0
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.001133,NaN,NaN,0,0.068165,NaN,1
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,0.987828,NaN,NaN,0,0.037619,NaN,0
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.037490,NaN,NaN,0,0.065417,NaN,1


In [ ]:
# 3. Calculate the Sharpe Ratio
# Recalculate volatility
stocks_df['volatility'] = stocks_df['Close'].rolling(30).std() * np.sqrt(252)

# Sharpe Ratio = (Expected Return - Risk-Free Rate) / Volatility
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [ ]:
stocks_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
23383,2.840,3.000,2.6400,2.875,63100,0.0,0.0,ROMA,2025,6,...,5.424529,NaN,NaN,3.1485,2.86180,1,0.125217,9.939668,0,0.541218
23384,2.850,2.935,2.7900,2.795,10200,0.0,0.0,ROMA,2025,6,...,5.008960,NaN,NaN,3.0870,2.89955,1,0.051878,9.469241,0,0.524219
23385,2.883,2.900,2.7300,2.790,33600,0.0,0.0,ROMA,2025,6,...,5.157116,NaN,NaN,2.9900,2.93555,1,0.060932,9.196931,0,0.555850
23386,2.960,3.120,2.7000,2.780,123300,0.0,0.0,ROMA,2025,6,...,5.325670,NaN,NaN,2.9110,2.97355,0,0.151079,9.001998,0,0.586611
23387,2.780,2.900,2.7013,2.800,104288,0.0,0.0,ROMA,2025,6,...,5.223880,NaN,NaN,2.8210,3.01405,0,0.070964,8.798129,0,0.588634


In [ ]:
stocks_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Ticker', 'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d',
       'growth_7d', 'growth_30d', 'growth_90d', 'growth_252d', 'growth_365d',
       'growth_future_30d', 'SMA10', 'SMA20', 'growing_moving_average',
       'high_minus_low_relative', 'volatility',
       'is_positive_growth_30d_future', 'Sharpe'],
      dtype='object')

In [ ]:
stocks_df['Date']

,Date
0,2024-05-23
1,2024-05-24
2,2024-05-28
3,2024-05-29
4,2024-05-30
...,...
23383,2025-06-16
23384,2025-06-17
23385,2025-06-18
23386,2025-06-20


In [ ]:
print(stocks_df['Date'].dtype)
print(pd.api.types.is_datetime64_any_dtype(stocks_df['Date'].dtype))

object
False


In [ ]:
# 4a. Filter to keep data only for ‘2025-06-06’
# 4b. Compute descriptive statistics for columns growth_252d and Sharpe

# Convert the Date to datetime if it's not already
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

# Filter the DataFrame for the trading day '2025-06-06'
filtered_stocks_df = stocks_df[stocks_df['Date'] == '2025-06-06']

# 5. What is the median Sharpe ratio for these 71 stocks?
descriptive_stats = filtered_stocks_df[['growth_252d', 'Sharpe']].describe()

# Display the descriptive statistics
print(descriptive_stats)

       growth_252d     Sharpe
count    71.000000  71.000000
mean      1.152898   0.288285
std       1.406017   0.519028
min       0.024970  -0.079677
25%       0.293422   0.041215
50%       0.758065   0.083768
75%       1.362736   0.311507
max       8.097413   2.835668


##Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**

(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)

**Goal:**

Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

**Steps:**

1. Start from the existing DataFrame from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

  Add 12 new columns:

  future_growth_1m, future_growth_2m, ..., future_growth_12m
  (Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)

  This logic is similar to historyPrices['growth_future_30d'] from Code Snippet 7, but extended to longer timeframes.

2. Determine the first trading day (min_date) for each ticker.
This is the earliest date in the data for each stock.

3. Join the data:
Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.
➤ You should end up with 75 records (one per IPO) with all 12 future_growth_... fields populated.

4. Compute descriptive statistics for the resulting DataFrame:
Use .describe() or similar to analyze each of the 12 columns:

  * future_growth_1m
  * future_growth_2m
  * ...
  * future_growth_12m

5. Determine the best holding period:

  * Find the number of months (1 to 12) where the average (mean) future growth is maximal.
  * This optimal month shows an uplift of >1% compared to all others.
  * Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

In [ ]:
# Sort stocks_df by Ticker and Date before calculating the future growth columns
holding_ipo = stocks_df.sort_values(by=['Ticker', 'Date'])

# 1. Add 12 new future growth columns
for months in range(1, 13):
    days = months * 21 # Assume 1 month = 21 trading days
    holding_ipo[f'future_growth_{months}m'] = holding_ipo.groupby('Ticker')['Close'].shift(-days) / holding_ipo['Close']

In [ ]:
holding_ipo.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
16396,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,AHR,2024,2,...,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
16397,12.245675,12.452591,12.226865,12.283297,1630300,0.0,0.0,AHR,2024,2,...,1.094691,1.156243,1.267090,1.324937,1.817033,1.951132,2.105590,2.256846,2.197812,2.230109
16398,12.236270,12.650102,12.179838,12.339727,2062100,0.0,0.0,AHR,2024,2,...,1.086582,1.157160,1.262084,1.352007,1.803991,1.927870,2.077638,2.219440,2.180527,2.207050
16399,13.120365,13.694088,12.320917,12.367943,1161200,0.0,0.0,AHR,2024,2,...,1.090294,1.151425,1.278880,1.390634,1.864410,1.953675,2.103896,2.231068,2.194798,2.278196
16400,12.367942,12.753559,12.236269,12.556048,1631700,0.0,0.0,AHR,2024,2,...,1.067100,1.131888,1.289178,1.399259,1.905472,1.933801,2.028534,2.218782,2.205361,2.222739


In [ ]:
len(holding_ipo)

23388

In [ ]:
# 2. Determine the first trading day (min_date) for each ticker
min_dates = holding_ipo.groupby('Ticker')['Date'].min().reset_index()# Add index
min_dates.columns = ['Ticker', 'min_date']

In [ ]:
min_dates.head()

,Ticker,min_date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22


In [ ]:
# 3. Join the min_dates DataFrame with the future growth data on ticker and date
# Filter holding_ipo to only include rows that match the min_date for each ticker
ipo_first_day_growth = pd.merge(
    min_dates, # Left DataFrame
    holding_ipo, # Right DataFrame
    left_on=['Ticker', 'min_date'], # The min_dates keys for the merge
    right_on=['Ticker', 'Date'], # The holding_ipo keys for the merge
    how='inner' # Only include rows where the merge keys exist in both DataFrames
)

In [ ]:
ipo_first_day_growth.head()

,Ticker,min_date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,AHR,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,2024,...,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,ALAB,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083


In [ ]:
ipo_first_day_growth.columns

Index(['Ticker', 'min_date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividends', 'Stock Splits', 'Year', 'Month', 'Weekday', 'Date',
       'growth_1d', 'growth_3d', 'growth_7d', 'growth_30d', 'growth_90d',
       'growth_252d', 'growth_365d', 'growth_future_30d', 'SMA10', 'SMA20',
       'growing_moving_average', 'high_minus_low_relative', 'volatility',
       'is_positive_growth_30d_future', 'Sharpe', 'future_growth_1m',
       'future_growth_2m', 'future_growth_3m', 'future_growth_4m',
       'future_growth_5m', 'future_growth_6m', 'future_growth_7m',
       'future_growth_8m', 'future_growth_9m', 'future_growth_10m',
       'future_growth_11m', 'future_growth_12m'],
      dtype='object')

In [ ]:
# Select only the relevant columns
future_growth_columns = [f'future_growth_{months}m' for months in range(1, 13)]
analysis_df = ipo_first_day_growth[['Ticker'] + future_growth_columns]

In [ ]:
analysis_df.head()

,Ticker,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,AHR,1.044629,1.061430,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,ALAB,1.039658,1.168951,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,0.772947,0.680193,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,1.294030,1.172388,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,0.713542,0.923958,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083


In [ ]:
# 4. Compute descriptive statistics for the resulting DataFrame
descriptive_stats = analysis_df.describe()
display(descriptive_stats)

,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,0.927259,0.940544,0.833825,0.825087,0.803769,0.864186,0.847150,0.832984,0.881780,0.917946,0.882535,0.900862
std,0.346261,0.574545,0.409763,0.401969,0.488349,0.653180,0.712944,0.762423,0.936957,0.911431,0.863183,0.892906
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.044086,0.043103,0.033144,0.037769,0.023674,0.038947
25%,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.634667
75%,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082457,1.049719,1.200830,1.106780,1.136392
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711


In [ ]:
# 5. Determine the best holding period
mean_growths = descriptive_stats.loc['mean']
optimal_months = mean_growths.idxmax()
optimal_growth_value = mean_growths.max()

print(f"Average future growth: ")
print(f"\n{mean_growths}")
print(f"\nBest holding period based on average growth: {optimal_months}")
print(f"Maximum average growth for the best holding period: {optimal_growth_value:.4f}")

Average future growth: 

future_growth_1m     0.927259
future_growth_2m     0.940544
future_growth_3m     0.833825
future_growth_4m     0.825087
future_growth_5m     0.803769
future_growth_6m     0.864186
future_growth_7m     0.847150
future_growth_8m     0.832984
future_growth_9m     0.881780
future_growth_10m    0.917946
future_growth_11m    0.882535
future_growth_12m    0.900862
Name: mean, dtype: float64

Best holding period based on average growth: future_growth_2m
Maximum average growth for the best holding period: 0.9405


##**Question 4: [Strategy] Simple RSI-Based Trading Strategy**

**What is the total profit (in dollar thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**

**Goal:**

Apply a simple rule-based trading strategy using the Relative Strength Index (RSI) technical indicator to identify oversold signals and calculate profits.

**Steps:**

1. Run the full notebook from Lecture 2 (33 stocks)

  * Ensure you can generate the merged DataFrame containing:
    * OHLCV data
    * Technical indicators
    * Macro indicators
  * Focus on getting RSI computed using Code Snippets 8 and 9.
  * This process is essential and will help during the capstone project.

2. ⚠️ IMPORTANT Please use this file to solve the Home Assignment (all next steps)

  Download precomputed data using this snippet:

  ```
    import gdown
    import pandas as pd

    file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
    gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
    df = pd.read_parquet("data.parquet", engine="pyarrow")
  ```

3. RSI Strategy Setup:

  * RSI is already available in the dataset as a field.
  * The threshold for oversold is defined as RSI < 25.

4. Filter the dataset by RSI and date:

  ```
    rsi_threshold = 25
    selected_df = df[
        (df['rsi'] < rsi_threshold) &
        (df['Date'] >= '2000-01-01') &
        (df['Date'] <= '2025-06-01')
    ]
  ```

5. Calculate Net Profit Over 25 Years:

  * Total number of trades: 1568
  * For each trade, you invest $1000
  * Use the 30-day forward return (growth_future_30d) to compute net earnings:
```
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
```
  * Final Answer:

    What is the net income in $K (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?



In [ ]:
# 2. Download the precomputed data
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")
df

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,3615.800049,3672.500000,3608.399902,3648.699951,1.678934e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,NaN,NaN,NaN,NaN,1.003714,1.020064,1.036306,1.156301,1.233323,1.597248
5687,3648.699951,3665.000000,3603.000000,3640.000000,2.013954e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.993181,0.989654,0.781299,0.842957,0.995927,1.011165,1.020634,1.162549,1.292217,1.570651
5688,3660.000000,3663.000000,3620.000000,3646.300049,1.293244e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.992203,1.000308,0.798376,0.886128,0.989061,0.988691,0.982898,1.135015,1.272691,1.578452
5689,3663.899902,3668.899902,3618.000000,3655.300049,1.972248e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.978792,0.991959,0.794034,0.863857,0.979958,0.965291,0.945990,1.120454,1.252080,1.563254


In [ ]:
# 4. Filter the dataset by RSI and date
rsi_threshold = 25 # Oversold condition
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

selected_df

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
3668,20.056772,20.114241,19.405453,19.673643,99915200.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,19.692798,19.807736,19.060636,19.309669,69037800.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3680,18.256067,18.332693,17.317403,17.336559,85374000.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3681,17.279087,17.336557,16.704395,16.991741,136453400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,17.010902,17.547282,16.934277,16.972589,81099400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,797.393417,806.609179,764.362578,769.289795,7251950.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.764911,0.515014,0.457056,0.336604,1.181878,1.234663,1.245515,0.610451,0.857647,1.514674
4406,774.673304,800.951963,758.522860,788.998840,6130185.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.753842,0.491171,0.431611,0.320580,1.001225,1.186226,1.114145,0.643468,0.862000,1.538415
4407,738.175133,739.041975,644.192390,660.662170,7308612.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.813666,0.496236,0.434706,0.322131,1.100520,1.035093,1.279557,0.663996,0.876243,1.595238
4408,698.550405,699.244329,621.756122,654.971985,7110384.0,10.0,0.0,LT.NS,2020,2020-03-01,...,0.802068,0.509668,0.437480,0.319412,1.049637,1.088881,1.288802,0.678603,0.925726,1.699390


In [ ]:
# 5. Calculate Net Profit Over 25 Years
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

# Convert net income to thousands of dollars
net_income_thousands = net_income / 1000
print(f"The net income in $K from the RSI-based oversold strategy is: {net_income_thousands:.2f}")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=8b2539b6-b43a-4017-8c87-a3c877b30282
To: /content/data.parquet
100%|██████████| 130M/130M [00:02<00:00, 62.5MB/s]


The net income in $K from the RSI-based oversold strategy is: 24.30
